# Interval based time series classification in sktime

Interval based approaches look at phase dependant intervals of the full series, calculating summary statistics from selected subseries to be used in classification.

Currently 5 univariate interval based approaches are implemented in sktime. Time Series Forest (TSF) \[1\], the Random Interval Spectral Ensemble (RISE) \[2\], Supervised Time Series Forest (STSF) \[3\], the Canonical Interval Forest (CIF) \[4\] and the Diverse Representation Canonical Interval Forest (DrCIF). Both CIF and DrCIF have multivariate capabilities.

In this notebook, we will demonstrate how to use these classifiers on the ItalyPowerDemand and BasicMotions datasets.

#### References:

\[1\] Deng, H., Runger, G., Tuv, E., & Vladimir, M. (2013). A time series forest for classification and feature extraction. Information Sciences, 239, 142-153.

\[2\] Flynn, M., Large, J., & Bagnall, T. (2019). The contract random interval spectral ensemble (c-RISE): the effect of contracting a classifier on accuracy. In International Conference on Hybrid Artificial Intelligence Systems (pp. 381-392). Springer, Cham.

\[3\] Cabello, N., Naghizade, E., Qi, J., & Kulik, L. (2020). Fast and Accurate Time Series Classification Through Supervised Interval Search. In IEEE International Conference on Data Mining.

\[4\] Middlehurst, M., Large, J., & Bagnall, A. (2020). The Canonical Interval Forest (CIF) Classifier for Time Series Classification. arXiv preprint arXiv:2008.09172.

\[5\] Lubba, C. H., Sethi, S. S., Knaute, P., Schultz, S. R., Fulcher, B. D., & Jones, N. S. (2019). catch22: CAnonical Time-series CHaracteristics. Data Mining and Knowledge Discovery, 33(6), 1821-1852.

## 1. Imports

In [136]:
import matplotlib.pyplot as plt

In [152]:
import pandas as pd

In [137]:
from sklearn import metrics

from sktime.classification.interval_based import (
    CanonicalIntervalForest,
    DrCIF,
    RandomIntervalSpectralEnsemble,
    SupervisedTimeSeriesForest,
    TimeSeriesForestClassifier,
)
from sktime.datasets import load_basic_motions, load_italy_power_demand

## 2. Load data

In [186]:
X_train, y_train = load_italy_power_demand(split="train", return_X_y=True)
X_test, y_test = load_italy_power_demand(split="test", return_X_y=True)
X_test = X_test[:50]
y_test = y_test[:50]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

X_train_mv, y_train_mv = load_basic_motions(split="train", return_X_y=True)
X_test_mv, y_test_mv = load_basic_motions(split="test", return_X_y=True)

X_train_mv = X_train_mv[:50]
y_train_mv = y_train_mv[:50]
X_test_mv = X_test_mv[:50]
y_test_mv = y_test_mv[:50]

print(X_train_mv.shape, y_train_mv.shape, X_test_mv.shape, y_test_mv.shape)

(67, 1) (67,) (50, 1) (50,)
(40, 6) (40,) (40, 6) (40,)


In [187]:
X_train

,dim_0
0,0 -0.710518 1 -1.183320 2 -1.372442 3...
1,0 -0.993009 1 -1.426786 2 -1.579884 3...
2,0 1.319067 1 0.569774 2 0.195128 3...
3,0 -0.812444 1 -1.157553 2 -1.416385 3...
4,0 -0.972840 1 -1.390518 2 -1.536705 3...
...,...
62,0 -1.037930 1 -1.301658 2 -1.469485 3...
63,0 -0.373579 1 -0.986632 2 -1.446422 3...
64,0 -1.159152 1 -1.301400 2 -1.524933 3...
65,0 -0.694919 1 -1.235829 2 -1.416133 3...


In [188]:
y_train

array(['1', '1', '2', '2', '1', '1', '2', '1', '1', '2', '2', '1', '1',
       '2', '1', '2', '1', '1', '2', '1', '1', '2', '1', '1', '1', '1',
       '1', '2', '2', '1', '1', '2', '2', '1', '2', '2', '1', '2', '1',
       '2', '1', '1', '2', '2', '1', '2', '2', '2', '2', '1', '1', '2',
       '2', '2', '1', '2', '2', '1', '1', '2', '2', '1', '1', '2', '1',
       '2', '2'], dtype='<U1')

In [189]:
def prepare_data(X_train):
    x = []
    for i in range(X_train.shape[0]):
        v = X_train.iloc[i].astype(str).values[0].split('\n')[:-1]
        x.append([float(name.split('   ')[1].replace(' ','')) for name in v])
    x_train = np.array(x)
    return pd.DataFrame(x_train)

In [190]:
X_train = prepare_data(X_train)
X_test = prepare_data(X_test)
y_train = y_train.astype(int)-1
y_test = y_test.astype(int)-1

In [191]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score

In [192]:
y_test

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0])

In [193]:
y_train

array([0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       1])

In [194]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
y_hat = forest.predict(X_test)
recall_score(y_test, y_hat)

0.9285714285714286

In [207]:
def get_stats(X_train):
    X_stat = pd.DataFrame()
    
    means = X_train.mean(axis=1)
    X_stat['mean'] = means
    
    std = X_train.std(axis=1)
    X_stat['std'] = std
    
    max_ = X_train.max(axis=1)
    X_stat['max_'] = max_
    
    min_ = X_train.min(axis=1)
    X_stat['min_'] = min_
    return X_stat

In [208]:
X_train_stat = get_stats(X_train)
X_test_stat = get_stats(X_test)

In [211]:
forest = RandomForestClassifier()
forest.fit(X_train_stat, y_train)
y_hat = forest.predict(X_test_stat)
recall_score(y_test, y_hat)

0.6428571428571429

In [214]:
X_train['max_'] = X_train.max(axis=1)
X_train['min_'] = X_train.min(axis=1)

X_test['max_'] = X_test.max(axis=1)
X_test['min_'] = X_test.min(axis=1)

In [215]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
y_hat = forest.predict(X_test)
recall_score(y_test, y_hat)

/Users/belyakov/miniforge3/envs/env_tf/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/belyakov/miniforge3/envs/env_tf/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


1.0

## 3. Time Series Forest (TSF)

TSF is an ensemble of tree classifiers built on the summary statistics of randomly selected intervals.
For each tree sqrt(series_length) intervals are randomly selected.
From each of these intervals the mean, standard deviation and slope is extracted from each time series and concatenated into a feature vector.
These new features are then used to build a tree, which is added to the ensemble.

In [103]:
X_train, y_train = load_italy_power_demand(split="train", return_X_y=True)

In [130]:
x_train.shape

(67, 24)

In [131]:
x_train = np.hstack((x_train, 
                     x_train.max(axis=1).reshape(-1,1),
                     x_train.min(axis=1).reshape(-1,1),
                     x_train.std(axis=1).reshape(-1,1),
                     x_train.mean(axis=1).reshape(-1,1)
                    ))

In [132]:
x_test = np.hstack((x_test, 
                     x_test.max(axis=1).reshape(-1,1),
                     x_test.min(axis=1).reshape(-1,1),
                     x_test.std(axis=1).reshape(-1,1),
                     x_test.mean(axis=1).reshape(-1,1)
                    ))

In [133]:
x_train.shape

(67, 28)

In [134]:
x_test.shape

(50, 28)

In [135]:
tsf = TimeSeriesForestClassifier(n_estimators=50, random_state=47)
tsf.fit(x_train, y_train)

tsf_preds = tsf.predict(x_test)
print("TSF Accuracy: " + str(metrics.accuracy_score(y_test, tsf_preds)))

TSF Accuracy: 1.0


In [21]:
tsf = TimeSeriesForestClassifier(n_estimators=50, random_state=47)
tsf.fit(X_train, y_train)

tsf_preds = tsf.predict(X_test)
print("TSF Accuracy: " + str(metrics.accuracy_score(y_test, tsf_preds)))

TSF Accuracy: 0.98


## 4. Random Interval Spectral Ensemble (RISE)

RISE is a tree based interval ensemble aimed at classifying audio data. Unlike TSF, it uses a single interval for each tree, and it uses spectral features rather than summary statistics.

In [4]:
rise = RandomIntervalSpectralEnsemble(n_estimators=50, random_state=47)
rise.fit(X_train, y_train)

rise_preds = rise.predict(X_test)
print("RISE Accuracy: " + str(metrics.accuracy_score(y_test, rise_preds)))

RISE Accuracy: 1.0


## 5. Supervised Time Series Forest (STSF)

STSF makes a number of adjustments from the original TSF algorithm. A supervised method of selecting intervals replaces random selection. Features are extracted from intervals generated from additional representations in periodogram and 1st order differences. Median, min, max and interquartile range are included in the summary statistics extracted.

In [5]:
stsf = SupervisedTimeSeriesForest(n_estimators=50, random_state=47)
stsf.fit(X_train, y_train)

stsf_preds = stsf.predict(X_test)
print("STSF Accuracy: " + str(metrics.accuracy_score(y_test, stsf_preds)))

STSF Accuracy: 0.96


## 6. Canonical Interval Forest (CIF)

CIF extends from the TSF algorithm. In addition to the 3 summary statistics used by TSF, CIF makes use of the features from the `Catch22` \[5\] transform.
To increase the diversity of the ensemble, the number of TSF and catch22 attributes is randomly subsampled per tree.

### Univariate

In [6]:
cif = CanonicalIntervalForest(n_estimators=50, att_subsample_size=8, random_state=47)
cif.fit(X_train, y_train)

cif_preds = cif.predict(X_test)
print("CIF Accuracy: " + str(metrics.accuracy_score(y_test, cif_preds)))

CIF Accuracy: 0.98


### Multivariate

In [7]:
cif_m = CanonicalIntervalForest(n_estimators=50, att_subsample_size=8, random_state=47)
cif_m.fit(X_train_mv, y_train_mv)

cif_m_preds = cif_m.predict(X_test_mv)
print("CIF Accuracy: " + str(metrics.accuracy_score(y_test_mv, cif_m_preds)))

CIF Accuracy: 1.0


## 6. Diverse Representation Canonical Interval Forest (DrCIF)

DrCIF makes use of the periodogram and differences representations used by STSF as well as the addition summary statistics in CIF.

### Univariate

In [8]:
drcif = DrCIF(n_estimators=5, att_subsample_size=10, random_state=47)
drcif.fit(X_train, y_train)

drcif_preds = drcif.predict(X_test)
print("DrCIF Accuracy: " + str(metrics.accuracy_score(y_test, drcif_preds)))

DrCIF Accuracy: 0.98


### Multivariate

In [9]:
drcif_m = DrCIF(n_estimators=5, att_subsample_size=10, random_state=47)
drcif_m.fit(X_train_mv, y_train_mv)

drcif_m_preds = drcif_m.predict(X_test_mv)
print("DrCIF Accuracy: " + str(metrics.accuracy_score(y_test_mv, drcif_m_preds)))

DrCIF Accuracy: 1.0
